In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

### calculating the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

In [3]:
nba_df = pd.read_csv("assets/nba.csv")

In [4]:
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [5]:
cities = pd.read_html("assets/wikipedia_data.html")[1]

In [6]:
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [7]:
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [8]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [15]:
population = cities[["Metropolitan area","Population (2016 est.)[8]"]]

In [16]:
population.head()

,Metropolitan area,Population (2016 est.)[8]
0,New York City,20153634
1,Los Angeles,13310447
2,San Francisco Bay Area,6657982
3,Chicago,9512999
4,Dallas–Fort Worth,7233323


In [17]:
population = population.copy()

In [18]:
population.loc[:, 'Metropolitan area'] = population['Metropolitan area'].str.strip()

In [19]:
population.head()

,Metropolitan area,Population (2016 est.)[8]
0,New York City,20153634
1,Los Angeles,13310447
2,San Francisco Bay Area,6657982
3,Chicago,9512999
4,Dallas–Fort Worth,7233323


In [20]:
population.rename(columns={'Population (2016 est.)[8]':'Population'},inplace=True)

In [22]:
population = population.set_index("Metropolitan area")

In [23]:
population.head()

,Population
Metropolitan area,
New York City,20153634
Los Angeles,13310447
San Francisco Bay Area,6657982
Chicago,9512999
Dallas–Fort Worth,7233323


In [24]:
cities["NBA"].replace(r"(.*)\[.*\].*|[-—]",r"\1",regex=True,inplace=True)

In [28]:
cities["NBA"].replace("—",np.nan,inplace=True)

In [30]:
cities["NBA"] = cities["NBA"].str.strip("—").str.strip(" ")

In [35]:
cities["NBA"].replace("",np.nan,inplace=True)

In [37]:
cities.dropna(inplace=True)

In [39]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [40]:
nba_cities = cities[["Metropolitan area","NBA"]].set_index("NBA")

In [42]:
nba_cities.head()

,Metropolitan area
NBA,
Knicks Nets,New York City
Lakers Clippers,Los Angeles
Warriors,San Francisco Bay Area
Bulls,Chicago
Mavericks,Dallas–Fort Worth


In [43]:
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [45]:
nba_df = nba_df[nba_df["year"]==2018].drop(["W/L%","GB","PS/G","PA/G","SRS","League","year"],axis=1)

In [46]:
nba_df.head()

,team,W,L
0,Toronto Raptors* (1),59,23
1,Boston Celtics* (2),55,27
2,Philadelphia 76ers* (3),52,30
3,Cleveland Cavaliers* (4),50,32
4,Indiana Pacers* (5),48,34


In [47]:
nba_df["team"].replace(r"(.*)[\*]|[\(].*",r"\1".strip(),regex=True,inplace=True)

In [48]:
nba_df.head()

,team,W,L
0,Toronto Raptors,59,23
1,Boston Celtics,55,27
2,Philadelphia 76ers,52,30
3,Cleveland Cavaliers,50,32
4,Indiana Pacers,48,34


In [50]:
nba_df[["team","W","L"]] =nba_df[["team","W","L"]].apply(lambda x: x.str.strip())

In [51]:
nba_df.head()

,team,W,L
0,Toronto Raptors,59,23
1,Boston Celtics,55,27
2,Philadelphia 76ers,52,30
3,Cleveland Cavaliers,50,32
4,Indiana Pacers,48,34


In [52]:
nba_df["area"] = nba_df["team"].str.split(" ").str[-1:]

In [54]:
nba_df["area"] = nba_df["area"].str[0]

In [57]:
nba_df.rename(columns={"area":"Area"},inplace=True)

In [58]:
nba_df.head()

,team,W,L,Area
0,Toronto Raptors,59,23,Raptors
1,Boston Celtics,55,27,Celtics
2,Philadelphia 76ers,52,30,76ers
3,Cleveland Cavaliers,50,32,Cavaliers
4,Indiana Pacers,48,34,Pacers


In [59]:
def get_area(team):
    for each in list(nba_cities.index.values):
        if team in each:
            return nba_cities.at[each,"Metropolitan area"]

In [60]:
nba_df["Area"] = nba_df["Area"].apply(lambda x:get_area(x))

In [61]:
nba_df.head()

,team,W,L,Area
0,Toronto Raptors,59,23,Toronto
1,Boston Celtics,55,27,Boston
2,Philadelphia 76ers,52,30,Philadelphia
3,Cleveland Cavaliers,50,32,Cleveland
4,Indiana Pacers,48,34,Indianapolis


In [71]:
nba_df[["W","L"]] = nba_df[["W","L"]].apply(pd.to_numeric,axis=1)

In [72]:
nba_df = nba_df.groupby("Area").sum(numeric_only=True)

In [73]:
nba_df

,W,L
Area,,
Atlanta,24,58
Boston,55,27
Charlotte,36,46
Chicago,27,55
Cleveland,50,32
Dallas–Fort Worth,24,58
Denver,46,36
Detroit,39,43
Houston,65,17


In [74]:
nba_df.eval("Ratio = W / (W + L)",inplace=True)

In [75]:
nba_df.head()

,W,L,Ratio
Area,,,
Atlanta,24,58,0.292683
Boston,55,27,0.670732
Charlotte,36,46,0.439024
Chicago,27,55,0.329268
Cleveland,50,32,0.609756


In [76]:
nba_df.drop(["W","L"],axis=1,inplace=True)

In [77]:
nba_df.head()

,Ratio
Area,
Atlanta,0.292683
Boston,0.670732
Charlotte,0.439024
Chicago,0.329268
Cleveland,0.609756


In [78]:
out_df = pd.merge(nba_df,population,how="inner",right_index=True,left_index=True)

In [79]:
out_df.head()

,Ratio,Population
Atlanta,0.292683,5789700
Boston,0.670732,4794447
Charlotte,0.439024,2474314
Chicago,0.329268,9512999
Cleveland,0.609756,2055612


In [80]:
out_df["Population"] = pd.to_numeric(out_df["Population"])

In [81]:
def nba_correlation():
    population_by_region = out_df["Population"]
    win_loss_by_region =out_df["Ratio"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2:Your Lists must be the Same Length"
    assert len(population_by_region) == 28, "Q2: There should be 28 Teams being Analysed for NBA"
    
    return stats.pearsonr(population_by_region,win_loss_by_region)[0]

In [82]:
nba_correlation()

-0.17657160252844611